# Chapter 04 — UMAP Clustering and Doublet analysis

## Objective

In this chapter, we load all single-nucleus RNA-seq datasets required for benchmarking the **Singulator + FACS** and **Singulator + LeviCell** protocols. We also verify metadata consistency, perform basic integrity checks, and prepare the data objects for downstream analysis.

This includes:

- Locating the data on the shared filesystem (Iris)
- Loading raw count matrices (e.g., `filtered_feature_bc_matrix.h5` from 10X)
- Loading data into `AnnData` objects

## Data Source

The data for this benchmarking project is stored on the **Iris** HPC filesystem under:

`/data1/collab002/sail/isabl/datalake/prod/010/collaborators/SAIL/projects/singulator_debris_removal_and/experiments`

We will be working with the data under identifier `MB-4027_*`


## Core Imports

In [1]:
import os
import scanpy as sc
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from typing import Callable, Dict, List, Union

## File Paths and Metadata

In [2]:
# Constants
DATA_DIR = "./data"
READ_ONLY_DIR = os.path.join(DATA_DIR, "read_only")
FIGURES_OUTPUT_DIR = os.path.join(DATA_DIR, "figures", "chapter_04_UMAP_clustering_doublet_analysis")

# Make sure figures output directory exists
os.makedirs(FIGURES_OUTPUT_DIR, exist_ok=True)

ANALYSIS_DIR = os.path.join(DATA_DIR, "analysis")
ADATA_DIR = os.path.join(ANALYSIS_DIR, "adatas")

# Make sure figures output directory exists
os.makedirs(FIGURES_OUTPUT_DIR, exist_ok=True)

# Sample metadata - in data/metadata.tsv
samples = {
    "SF_N":  ("MB-4027_SF_N",  "Normal Colon", "Singulator+FACS"),
    "SL_N":  ("MB-4027_SL_N",  "Normal Colon", "Singulator+LeviCell"),
    "SF_T":  ("MB-4027_SF_T",  "Tumor Colon",  "Singulator+FACS"),
    "SL_T":  ("MB-4027_SL_T",  "Tumor Colon",  "Singulator+LeviCell"),
    "SF_LN": ("MB-4027_SF_LN", "Normal Liver", "Singulator+FACS"),
    "SL_LN": ("MB-4027_SL_LN", "Normal Liver", "Singulator+LeviCell"),
}

# Color palette for plotting
protocol_color_palette = {
    "Singulator+FACS": "#AEC6CF",
    "Singulator+LeviCell": "#FFDAB9",
}


## Read in AnnDatas

In [3]:
# Load AnnData objects
adatas = {}
adata_metadata = {}
adata_tissues = []

for key, (folder, tissue, protocol) in samples.items():
    file_path = os.path.join(ADATA_DIR, f"{key}_adata.h5ad")
    adata = sc.read_h5ad(file_path)
    adatas[key] = adata
    adata_metadata[key] = (tissue, protocol)
    adata_tissues.append(tissue)
    print(f"{key}: {adata}")

SF_N: AnnData object with n_obs × n_vars = 7179 × 38606
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mitochondrial', 'pct_counts_mitochondrial', 'total_counts_ribosomal', 'pct_counts_ribosomal', 'total_counts_apoptosis', 'pct_counts_apoptosis', 'total_counts_housekeeping', 'pct_counts_housekeeping'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'mitochondrial', 'ribosomal', 'apoptosis', 'housekeeping'
SL_N: AnnData object with n_obs × n_vars = 7929 × 38606
    obs: 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'total_counts_mitochondrial', 'pct_counts_mitochondrial', 'total_counts_ribosomal', 'pct_counts_ribosomal', 'total_counts_apoptosis', 'pct_counts_apoptosis', 'total_counts_housekeeping', 'pct_counts_housekeeping'
    var: 'gene_ids', 'feature_types', 'genome'

## Doublet Detection

In [6]:
def run_scrublet(adata : sc.AnnData) -> None:
    """
    Run Scrublet doublet detection on the given AnnData object.
    
    Parameters:
    adata (sc.AnnData): The AnnData object to analyze.
    """
    sc.pp.scrublet(adata, sim_doublet_ratio=2.0, expected_doublet_rate=0.06, 
                knn_dist_metric='euclidean', log_transform=True, n_prin_comps=30, 
                random_state=0) # From Roshan's workshop
    
for key, adata in adatas.items():
    print(f"Running Scrublet on {key}...")
    run_scrublet(adata)

Running Scrublet on SF_N...


adata.X seems to be already log-transformed.


Running Scrublet on SL_N...


adata.X seems to be already log-transformed.


Running Scrublet on SF_T...


...

adata.X seems to be already log-transformed.


Running Scrublet on SF_LN...


adata.X seems to be already log-transformed.


Running Scrublet on SL_LN...


adata.X seems to be already log-transformed.


## Preprocessing and Graph Computation

In [5]:
# Use downsampled reads for analysis
for key, adata in adatas.items():
    adata.layers["raw_data"] = adata.X.copy()
    adata.X = adata.layers["downsampled_umi"].copy()
    sc.pp.filter_cells(adata, min_counts=1) # Ensure no cells with zero counts
    sc.pp.normalize_total(adata, target_sum=None, inplace=True) # From both Roshan and sc-best-practices
    sc.pp.log1p(adata)

# Combine adatas by tissue
combined_by_tissue = {}
for tissue in adata_tissues:
    tissue_adatas = []
    for key, (tissue_name, protocol) in adata_metadata.items():
        if tissue_name == tissue:
            adata = adatas[key].copy()
            adata.obs["protocol"] = protocol
            adata.obs["tissue"] = tissue
            adata.obs["sample"] = key
            tissue_adatas.append(adata)

    combined = sc.concat(
        tissue_adatas, 
        label="sample", 
        keys=[a.obs['sample'].iloc[0] for a in tissue_adatas]
    )

    # Make obs and var names unique
    combined.obs_names_make_unique()
    combined.var_names_make_unique()
    
    # Filter cells and genes
    sc.pp.filter_cells(combined, min_genes=20) # From (https://www.sc-best-practices.org/preprocessing_visualization/quality_control.html)
    sc.pp.filter_genes(combined, min_cells=np.exp(4)) # From Roshan's workshop
    
    # Compute graphs
    sc.pp.highly_variable_genes(
        combined, 
        flavor="seurat_v3", 
        n_top_genes=4000, 
        batch_key="protocol", 
        layer="raw_data"
    ) # Using batch_key to account for protocol differences - want to keep genes that are variable across both protocols
    sc.pp.pca(combined, n_comps=None, use_highly_variable=True, random_state=0) #  Roshan's workshop used 100 comps, but this took long - if set to None, "Defaults to 50, or 1 - minimum dimension size of selected representation."
    sc.pp.neighbors(combined, n_neighbors=30, use_rep='X_pca', metric='euclidean', random_state=0) # From Roshan's workshop

    # Compute UMAP and clustering
    sc.tl.umap(combined, min_dist=0.1, random_state=0) # From Roshan's workshop
    sc.tl.leiden(combined, resolution = 1, random_state=0) # From Roshan's workshop

    combined_by_tissue[tissue] = combined


KeyError: 'downsampled_umi'

 ## Plot UMAPs by Protocol and Library Size


### Plotting Functions

In [ ]:
def plot_umap_by_obs_feature(
    combined_by_tissue: Dict[str, sc.AnnData],
    feature: str,
    color_palette: Union[Dict[str, str], str] = "viridis",
    title: str = None,
    size: float = 8.0,
    alpha: float = 0.8
) -> plt.Figure:
    """
    Plots UMAPs for each tissue colored by a feature in .obs.

    Args:
        combined_by_tissue (dict): Dictionary of AnnData objects per tissue.
        feature (str): Column in .obs to plot (categorical or continuous).
        color_palette (dict or str): Dict for categorical features or colormap name for continuous.
        title (str): Figure title.
        size (float): Dot size.
        alpha (float): Dot transparency.

    Returns:
        plt.Figure: The generated figure.
    """
    sns.set_theme(style="white")
    sample_tissues = set(combined_by_tissue.keys())
    n_tissues = len(sample_tissues)
    fig, axes = plt.subplots(1, n_tissues, figsize=(6 * n_tissues, 5), squeeze=False)

    for i, tissue in enumerate(sample_tissues):
        ax = axes[0, i]
        adata = combined_by_tissue[tissue]

        coords = adata.obsm["X_umap"]
        values = adata.obs[feature].copy()

        # Shuffle to avoid overplotting
        shuffled_idx = np.random.permutation(adata.n_obs)
        coords = coords[shuffled_idx]
        values = values.iloc[shuffled_idx]

        if isinstance(color_palette, dict) or values.dtype.name == "category" or values.dtype == object:
            # Categorical case
            sns.scatterplot(
                x=coords[:, 0], y=coords[:, 1],
                hue=values,
                palette=color_palette if isinstance(color_palette, dict) else "tab20",
                ax=ax, s=size, alpha=alpha, linewidth=0
            )
            ax.legend_.remove()  # suppress per-plot legends; add a global legend later
        else:
            # Continuous case
            scatter = ax.scatter(
                coords[:, 0], coords[:, 1],
                c=values,
                cmap=color_palette,
                s=size,
                alpha=alpha,
                linewidths=0
            )
            plt.colorbar(scatter, ax=ax, fraction=0.046, pad=0.04)

        ax.set_title(tissue)
        ax.set_xlabel("UMAP1")
        ax.set_ylabel("UMAP2" if i == 0 else "")
        ax.set_xticks([])
        ax.set_yticks([])

    if isinstance(color_palette, dict) or values.dtype.name == "category" or values.dtype == object:
        # Global categorical legend
        unique_vals = sorted(values.dropna().unique())
        legend_elements = [
            mpl.patches.Patch(facecolor=color_palette.get(val, "gray"), label=val)
            for val in unique_vals
        ]
        fig.legend(
            handles=legend_elements,
            title=feature,
            loc="upper right",
            bbox_to_anchor=(1.05, 1.05),
            fontsize='small',
            title_fontsize='medium'
        )

    plt.suptitle(title or f"UMAP Colored by {feature}", fontsize=16, weight="bold")
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    return fig

### Plot by Protocol

In [ ]:
# Plot UMAP colored by protocol
fig = plot_umap_by_obs_feature(
    combined_by_tissue = combined_by_tissue,
    feature = "protocol",
    color_palette = protocol_color_palette,
)

# Save the figure
fig.savefig(os.path.join(FIGURES_OUTPUT_DIR, "umap_by_protocol.png"), bbox_inches="tight")

### Plot by Library Size

In [ ]:
# Plot UMAP colored by library szie
plot_umap_by_obs_feature(
    combined_by_tissue = combined_by_tissue,
    feature = "total_counts",
    color_palette = "viridis",
)

# Save the figure
fig.savefig(os.path.join(FIGURES_OUTPUT_DIR, "umap_by_library_size.png"), bbox_inches="tight")

### Plot by Leiden Cluster

In [ ]:
# Plot UMAP colored by leiden cluster
plot_umap_by_obs_feature(
    combined_by_tissue = combined_by_tissue,
    feature = "leiden",
    color_palette = "tab20",
)

# Save the figure
fig.savefig(os.path.join(FIGURES_OUTPUT_DIR, "umap_by_leiden.png"), bbox_inches="tight")

In [ ]:
# Function to plot a scalar metric from multiple AnnData objects and create a subplot for each unique tissue.

def adaptive_formatter(x, _):
    """
    Formats large numeric values for axis tick labels using adaptive human-readable notation.

    - Values ≥ 1,000,000 are shown with 'M' (e.g., 2,500,000 → '2.5M')
    - Values < 1,000,000 are shown as plain integers (e.g., 42,000 → '42000')
    """
    if x >= 1e6:
        return f"{x / 1e6:.1f}M"
    else:
        return f"{int(x)}"

def plot_adata_scalar_metric(
    adatas: dict,
    adata_metadata: dict,
    metric_func: Callable,
    metric_label: str,
    protocol_color_palette: dict,
) -> None:
    """
    Plots a scalar metric computed on each AnnData object, grouped by tissue and protocol. Will create as many subplots as there are unique tissues.
    The metric function should return a scalar value for each AnnData object.
    The metric label is used for the Y-axis and title, and the color palette maps protocol names to colors.
    The function uses adaptive formatting for the Y-axis to handle large numbers (e.g., millions and thousands).
    It also annotates the bars with the actual values for clarity.

    Args:
        adatas: dict of AnnData objects keyed by sample ID
        adata_metadata: dict of (tissue, protocol) tuples keyed by sample ID
        metric_func: function that takes an AnnData object and returns a scalar
        metric_label: label to show on the Y-axis and title (e.g., "Total Reads")
        protocol_color_palette: mapping of protocol name to color
    Returns:
        fig: matplotlib figure object containing the plot
    """
    # Assemble data
    rows = []
    for key, adata in adatas.items():
        tissue, protocol = adata_metadata[key]
        value = metric_func(adata)
        rows.append({
            "Tissue": tissue,
            "Protocol": protocol,
            "Value": value
        })
    df = pd.DataFrame(rows)

    # Determine unique tissues (to define subplots)
    unique_tissues = df["Tissue"].unique()

    # Plot setup
    sns.set_theme(style="white", font_scale=1.1)
    fig, axes = plt.subplots(1, len(unique_tissues), figsize=(5 * len(unique_tissues), 5), sharey=True)

    if len(unique_tissues) == 1:
        axes = [axes]  # Ensure axes is always iterable

    for i, tissue in enumerate(unique_tissues):
        ax = axes[i]
        sns.barplot(
            data=df[df["Tissue"] == tissue],
            x="Protocol",
            y="Value",
            hue="Protocol",
            palette=protocol_color_palette,
            legend=False,
            ax=ax
        )
        ax.set_title(tissue, fontsize=13, weight="bold")
        ax.set_xlabel("")
        ax.set_ylabel(metric_label if i == 0 else "")
        ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(adaptive_formatter))

        for p in ax.patches:
            val = p.get_height()
            ax.annotate(f"{int(val):,}",
                        (p.get_x() + p.get_width() / 2., val),
                        ha="center", va="bottom", fontsize=10, weight="bold")

    plt.suptitle(f"{metric_label} by Protocol Across Tissues", fontsize=16, weight="bold")
    plt.tight_layout(rect=[0, 0, 1, 0.95])
    legend_elements = [mpl.patches.Patch(facecolor=color, label=label) for label, color in protocol_color_palette.items()]
    fig.legend(handles=legend_elements, title="Protocol", loc="upper right", bbox_to_anchor=(1.05, 1.05))
    return fig

### Clusters by Protocol

In [ ]:
# source ids per cluster
df_cluster_count = adata_filtered.obs.groupby(["source_id", "leiden_granular_2"], observed=True).size().reset_index()
df_wide = df_cluster_count.pivot(index="leiden_granular_2", columns="source_id", values=0).reset_index()
df_wide.loc[:, "pool1_norm"] = df_wide.pool1 / df_wide[["pool1", "pool2"]].sum(axis=1)
df_wide.loc[:, "pool2_norm"] = 1

# make a stacked barplot of
ax = sns.barplot(df_wide, x="leiden_granular_2", y="pool2_norm", label="pool 2")
ax = sns.barplot(df_wide, x="leiden_granular_2", y="pool1_norm", label="pool 1")
ax.set_title("Cluster Proportions")
plt.legend(bbox_to_anchor=(1.2, 1), loc="upper right")

### Plot by Doublets

In [ ]:
# Plot UMAP colored by predicted doublets
plot_umap_by_obs_feature(
    combined_by_tissue = combined_by_tissue,
    feature = "predicted_doublet",
    color_palette = "rocket",
)

# Save the figure
fig.savefig(os.path.join(FIGURES_OUTPUT_DIR, "umap_by_predicted_doublets.png"), bbox_inches="tight")

NameError: name 'plot_umap_by_obs_feature' is not defined

### Plot by Apoptotic Gene Percentage

In [ ]:
# Plot UMAP colored by total counts of apoptosis genes
plot_umap_by_obs_feature(
    combined_by_tissue = combined_by_tissue,
    feature = "total_counts_apoptosis",
    color_palette = "inferno",
)

# Save the figure
fig.savefig(os.path.join(FIGURES_OUTPUT_DIR, "umap_by_total_counts_apoptosis.png"), bbox_inches="tight")

## Cell Typing

### Normal Liver

#### CellTypist

In [ ]:
import celltypist

cell_predictions = celltypist.annotate(
    combined_by_tissue["Normal Liver"],
    model = 'Healthy_Human_Liver.pkl',
    majority_voting = True,
    over_clustering='leiden'
)

cell_predictions_adata = cell_predictions.to_adata()

majority_voting_output_path = os.path.join(FIGURES_OUTPUT_DIR, "majority_voting_output.png")

sc.pl.umap(cell_predictions_adata, color = 'predicted_labels')
sc.pl.umap(cell_predictions_adata, color = 'majority_voting', title = "Cell Typist Majority Voting (Normal Liver) - using Healthy_Human_Liver.pkl", save = majority_voting_output_path)
sc.pl.umap(cell_predictions_adata, color = 'conf_score')


#### Marker Genes

In [ ]:
liver_marker_dict = {
    "Hepatocytes": ["APOB", "CYP3A4"], # https://www.nature.com/articles/s42003-022-04046-9
    "Cholangiocytes": ["KRT19", "CFTR"], # https://pmc.ncbi.nlm.nih.gov/articles/PMC4315871/
    "Endothelial cells": ["PECAM1", "CDH5", "LDB2", "PTPRB"], # https://www.nature.com/articles/s41467-022-30633-9, CELL x GENE
    "Fibroblasts": ["COL1A2", "COL5A2", "COL6A3"], # https://pmc.ncbi.nlm.nih.gov/articles/PMC7444611/, CELL x Gene
    "Macrophages": ["CD68", "C1QA", "C1QB", "C1QC"], #https://pmc.ncbi.nlm.nih.gov/articles/PMC10153153/, https://pmc.ncbi.nlm.nih.gov/articles/PMC10849641/
    "Resident NK": ["KLRD1", "NCAM1"], #https://www.ncbi.nlm.nih.gov/gene/3824, https://www.frontiersin.org/journals/immunology/articles/10.3389/fimmu.2017.00892/full
    "T cells": ["CD3D", "CD3E","CD3G", "CD2", "PTPRC"], # https://pubmed.ncbi.nlm.nih.gov/39029632/
    "B cells": ["CD19", "CD79A", "CD79B", "PTPRC"], # https://genomebiology.biomedcentral.com/articles/10.1186/s13059-020-02064-6?utm_source=chatgpt.com
}

#### Plot Dotplot

In [ ]:
liver_dp = sc.pl.dotplot(
    adata,
    var_names=liver_marker_dict,
    groupby="majority_voting",
    standard_scale="var",
    color_map="Reds",
    figsize=(10, 6),
    dendrogram=True,
    show=False
)

# Set the title on the figure
liver_dp.fig.suptitle("Liver Marker Gene Expression by Majority Voting", fontsize=16, weight="bold")

# Adjust layout to accommodate the title
liver_dp.fig.tight_layout(rect=[0, 0, 1, 0.95])

# Save the figure
liver_dp.savefig(os.path.join(FIGURES_OUTPUT_DIR, "normal_liver_marker_gene_dotplot.png"))


#### Plot Heatmap

In [ ]:
import os

# Create and store the heatmap object
liver_hm = sc.pl.heatmap(
    adata,
    var_names=liver_marker_dict,
    groupby="majority_voting",
    use_raw=False,
    standard_scale="var",
    cmap="Reds",
    swap_axes=True,
    show_gene_labels=True,
    var_group_rotation=90,
    var_group_labels=liver_marker_dict.keys(),
    dendrogram=True,
    figsize=(12, 10),
    show=False  # Prevent auto display
)

# Add title
liver_hm.fig.suptitle("Liver Marker Gene Expression Heatmap by Majority Voting", fontsize=16, weight="bold")

# Adjust layout to make room for title
liver_hm.fig.tight_layout(rect=[0, 0, 1, 0.95])

# Save the figure
liver_hm.savefig(os.path.join(FIGURES_OUTPUT_DIR, "normal_liver_marker_gene_heatmap.png"))


### Normal Colon

In [ ]:
colon_marker_dict = {
    "Epithelial" : ["EPCAM", "KRT8", "KRT18"],
    "Stromal" : ["COL6A2", "COL6A1", "PLVAP", "COL1A1"],
    "Immune" : ["PTPRC", "CD3E", "CD2", "CD3D", "CD3G", "FCER1G", "CSF1R", "CD14", "CD68", "CD38"],
}